<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="ntnu_logo.png" WIDTH=250 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Example 3:</ins> Python and Hydraulic Models
    
*Developed by David Steffelbauer*

<hr style="border:0.2px solid black"> </hr>

    
## Hydraulic Modelling with Python using WNTR

This notebook will show us the basic functionality of the WNTR package

### Installing packages like `WNTR`

Here is a link on the installation guide for `wntr` ($\rightarrow$[Installation](https://wntr.readthedocs.io/en/stable/installation.html)). 

The cool thing about Jupyter notebooks is that you can also install Python packages from here. So there is no need to open a terminal. You can just run the terminal commands by simply adding a `!` before the line. For example, instead of 

`pip install wntr`

in a terminal, we would type

`!pip install wntr`.

However, there might appear some problems, if you install a package like that (e.g. different Python versions installed on your machine).

There is a workaround, how to correctly install the `wntr` package without running into some errors. You can find detailed information about this issue in this excellent [blogpost](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install wntr

Now we can savely import the `wntr` package

In [ ]:
import wntr

Let us also import some other packages that will be useful for this notebook. This packages are the same as we used in the last lecture, when we had a look at the basic functionality of `Python`.

In [ ]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

### Loading EPANET input files to WNTR

First, we have to load the L-Town Epanet model into `wntr`. For that reason we use the [`wntr.network.WaterNetworkModel`](https://wntr.readthedocs.io/en/stable/apidoc/wntr.network.model.html#wntr.network.model.WaterNetworkModel) function.

In [ ]:
# Create a water network model
inp_file = 'L-Town.inp'

wn = wntr.network.WaterNetworkModel(inp_file)

Now the variable `wn` has all necessary information that was already contained in the Epanet input file. 

### Query information from Water Network Models

Next step, let's check if the overview information of the L-Town case study is correct that the water utility gave us. This is also good practice to see, if the EPANET input file is in the correct units. Furthermore, it is a good exercise to get familiar with the network.

Here is a reminder of the information:
* 43 km pipe length
* Map dimensions: 1.5 km heigth, 2.7 km width
* 782 Junctions, 2 Reservoirs, 1 Tank
* 905 Pipes, 1 Pump, 3 PRV
* Pipe segments of $\approx$50 meters
* $\approx$10.000 people

We can get specific information of different attributes with WNTR's query functions. There exist two different types of elements in WDS, nodes and links. Consequently, there are two different query functions:  
* [`query_node_attribute`](https://wntr.readthedocs.io/en/stable/apidoc/wntr.network.model.html#wntr.network.model.WaterNetworkModel.query_node_attribute) for nodes
* [`query_link_attribute`](https://wntr.readthedocs.io/en/stable/apidoc/wntr.network.model.html#wntr.network.model.WaterNetworkModel.query_link_attribute) for links

The total pipe length of the network is around 42 km. Let's see ourselves. We look for the lengths as an attribute of pipes, that is why we use the query function for links. The output is a Pandas DataFrame. 

In [ ]:
pipe_lengths = wn.query_link_attribute('length')
print(pipe_lengths)

Hence, we can build the sum over it. The result is in meters, dividing by 1000 transforms this to km. Morevover, we use Python3's [`f-strings`](https://realpython.com/python-f-strings/) to generate a nice output.

In [ ]:
total_length = pipe_lengths.sum()/1000
# print(total_length)
print(f'The total length of the pipes is {total_length:.1f} km.')

Second, we want to check the Map dimensions. We can use the coordinates for that, which are node attributes. The coordinates are a tuple of x and y coordinates. We transform that to a DataFrame containing two columns, 'x' and 'y', with a combination of Pandas [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) funciton and a Python [`lambda`](https://realpython.com/python-lambda/) function. Furthermore, we delete the original columns that contains the tuple with Pandas [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) method.

In [ ]:
coords = wn.query_node_attribute('coordinates')

coords = pd.DataFrame(coords)
coords['x'] = coords[0].apply(lambda x: x[0])
coords['y'] = coords[0].apply(lambda x: x[1])

coords = coords.drop(0, axis=1)
print(coords)

The Map dimensions height is given by the maximum difference in the y coordinates, from the biggest to the smallest value. We can use min and max for that. The same is true for the width and the x coordinates. Again, the units are meters. So we divide it by 1000 to get km and print a fancy `f-string` to show our result. Note the `\n` in the `f-string`...this indicates a line break. Characters with a backslash are called escape charakters ($\rightarrow$[link](https://www.w3schools.com/python/gloss_python_escape_characters.asp))

In [ ]:
width = (coords['x'].max() - coords['x'].min())/1000
height = (coords['y'].max() - coords['y'].min())/1000

print(f'Map dimensions:\n {height:.1f} km height, {width:.1f} km width')

In [ ]:
coords.describe()

How many elements are in the Network model. There is a handy function for that, the developers of WNTR just overloaded the Pandas describe function:

In [ ]:
wn.describe()

There are 909 links, put how many are pipes, how many pumps and how many are valves. We can use properties like `pipe_name_list` to find that out. Functions like `pipe_name_list`, as the name already indicates, give us a list of the names of all pipes.

In [ ]:
wn.pipe_name_list

With the build-in [`len`](https://docs.python.org/3/library/functions.html#len) functions, we can see, how many elements this list has.

In [ ]:
num_pipes = len(wn.pipe_name_list)
num_valves = len(wn.valve_name_list)
num_pumps = len(wn.pump_name_list)

print(f'{num_pipes:.0f} Pipes, {num_pumps:.0f} Pumps, {num_valves:.0f} PRV')

WNTR has also special functions like [`num_valves`](https://wntr.readthedocs.io/en/latest/apidoc/wntr.network.model.html?highlight=num_valves#wntr.network.model.WaterNetworkModel.num_valves), so you don't have to use `len` at all.

In [ ]:
print(f'{wn.num_pipes:.0f} Pipes, {wn.num_pumps:.0f} Pumps, {wn.num_valves:.0f} PRV')

Let's go on and check if the lengths are all around 50 meter. We already know how to get the lengths. This time we are not interested in the sum of the lengths, but the statistics behind it. 

In [ ]:
length = wn.query_link_attribute('length')
print(length)

The pandas [`describe`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) function can be used to get a good overview.

In [ ]:
length.describe()

We can also plot the statistics of the pipe segments in form of a histogram with pandas [`hist`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.hist.html#pandas.Series.hist) function:

In [ ]:
length.hist(bins=50)
plt.xlabel('lenghts of pipe segments (m)', fontsize=20)
plt.ylabel('number of pipes', fontsize=20)
plt.axvline(length.mean(), color='k', linewidth=2, linestyle='--')
plt.text(length.mean()-1, 90, f'mean={length.mean():.2f}', horizontalalignment='right', fontsize=14)

Besides the axis labelling functions that we already know ([`xlabel`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.xlabel.html), [`ylabel`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.ylabel.html)), we used here two additional Matplotlib functions. [`text`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.text.html) let's us annotate figures, [`axvline`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.axvline.html) draws a vertical line in our graph.

Last but not least, we want to check if the number of people that are supplied by the WDS is correct. Unfortunately, this information is not contained in the EPANET file. But we can use the demand as a proxy to check this information, by estimating the number of people from the daily water demand. The base demand in EPANET files is usually taken from billing information and shows the average consumption over multiple years. Note, that this is not always true, it depends on the different policies and practices in different countries!

The demand is an attribute of junctions, so we already know to extract this information from the EPANET file:

In [ ]:
demand = wn.query_node_attribute('base_demand')

Be careful, the units in WNTR and in the EPANET file may be different. WNTR converts all the units to this set of units [$\rightarrow$link](https://wntr.readthedocs.io/en/stable/units.html), despite that the units in the EPANET inpfile are different. You can see the original units in the `options` of the Water Network Model:

In [ ]:
wn.options.hydraulic.inpfile_units
wntr.__version__

That means that the demand is stored in $m^3/s$. Let's check what the total daily demand is in liters:

In [ ]:
total_daily_demand = demand.sum()*3600*24
print(f'The total daily demand is {total_daily_demand:.0f} cubic meters')

Let's calculate what that means in liters per person:

In [ ]:
ppdd = total_daily_demand * 1000 / 10000
print(f'Daily demand per person: {ppdd:.1f} Liters')

Let's check that, if that is correct by clicking this [link](http://letmegooglethat.com/?q=water+demand+cyprus).

### Plotting networks

In [ ]:
elevation = wn.query_node_attribute('elevation')

diameter = wn.query_link_attribute('diameter')*1000

wntr.graphics.plot_network(wn, link_attribute=diameter, node_size=0, link_colorbar_label='Diameter (mm)')

print(type(wn))
wntr.graphics.plot_network(wn, node_size=20, 
                           node_attribute=elevation, 
                           link_attribute=diameter,
                           node_colorbar_label='Elev (m)',
                           link_colorbar_label='Diam (m)'
                          )    

In [ ]:
def xsquared(x):
    return x**2

xsquared(6)

In [ ]:
wntr.graphics.plot_interactive_network(wn, node_attribute=elevation, filename='L-Town.html', auto_open=True, node_cmap='Plasma')

In [ ]:
from IPython.display import IFrame
IFrame(src='./L-Town.html', width=700, height=600)

### Running Simulations

In [ ]:
# Create a water network model
inp_file = 'L-town.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

In [ ]:
sim = wntr.sim.EpanetSimulator(wn)

results = sim.run_sim()



In [ ]:
q = results.link['flowrate']

In [ ]:
Q = results.node['demand']['n5']

In [ ]:
Q.plot()

In [ ]:
sim = wntr.sim.EpanetSimulator(wn)

results = sim.run_sim()
before = results.node['pressure']['n517']


for name, j in wn.junctions():
    #     j.base_demand = j.base_demand * 1.2
    j.demand_timeseries_list[0].base_value = j.base_demand * 1.2
    
    
results = sim.run_sim()
after = results.node['pressure']['n517']

In [ ]:
before.plot()
after.plot()

In [ ]:
(after - before).plot()

In [ ]:
before.index = pd.to_datetime(before.index, unit='s', origin='2021-01-27')

In [ ]:
plt.plot(before)